## To explore the GBDT.compute_loss

Binomial loss

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from random import sample
from math import exp, log

from gbdt.data import DataSet
from gbdt.model import GBDT
from gbdt.model import BinomialDeviance, RegressionLossFunction
from gbdt.tree import construct_decision_tree, Tree
from gbdt.tree import MSE

In [3]:
dataset = DataSet('data/credit.data.csv')
dataset.get_label_size()

2

#### The initial properties

In [4]:
max_iter = 20
sample_rate = 0.8
learn_rate = 0.5
max_depth = 7
loss_type = 'binary-classification'
split_points = 0
trees = dict()

#### Loss function: Binary classification
`GBDT.model.fit`: 

when `loss_type == 'binary-classification'`


In [5]:
loss = BinomialDeviance(n_classes=dataset.get_label_size())
loss.K

1

In [6]:
f = dict()
loss.initialize(f, dataset)

In [7]:
train_data = dataset.get_instances_idset()
# train_data is a set, which contains IDs from 1 to 653

In [8]:
# define compute_loss                
def compute_loss(dataset, subset, f):
    total_loss = 0.0
    if loss.K == 1:
        print(loss.K)
        for id in dataset.get_instances_idset():
            y_i = dataset.get_instance(id)['label']
            f_value = f[id]
            p_1 = 1/(1+exp(-2*f_value))
            try:
                total_loss -= ((1+y_i)*log(p_1)/2) + ((1-y_i)*log(1-p_1)/2) # Here we will explain deeper!
            except ValueError as e:
                print(y_i, p_1)
    else:
        for id in dataset.get_instances_idset():
            instance = dataset.get_instance(id)
            f_values = f[id]
            exp_values = {}
            for label in f_values:
                print('label is:', label)
                exp_values[label] = exp(f_values[label])
            probs = {}
            for label in f_values:
                probs[label] = exp_values[label]/sum(exp_values.values())
                # 预测的越准确则log(probs[instance["label"]])越接近0 loss也就越小
            total_loss -= log(probs[instance["label"]])
    return total_loss/dataset.size()


`GBDT.compute_loss`：

如果`loss.K==1`,对应的是log loss，此时loss的计算公式为$loss = loss - likelihood$
$$likelihood = \bigg(\frac{1+y_i}{2}\log p + \frac{1-y_i}{2}\log(1-p)\bigg)$$ 
其中，$f = f[id], p = \frac{1}{1+\exp(-2f)}$.

回忆：在`GBDT.model.BinomialDeviance.compute_residual`中，定义loss为
$$loss = \frac{2y_i}{1+\exp{(-2y_if)}}$$

#### 问题：这两个loss等价嘛？
我们从第一个表达式推到第二个：

第一个loss是来自于logistic regression的log loss，label的取值是0,1；第二个loss是来自于Friedman的文章，label的取值是-1,1。所以需要将0,1取值变换为-1,1取值：

$y = 2Y-1$，如果$Y\in\{0,1\}$，则$y\in\{-1,1\}$映射

令$p = P(y=1|x)=\frac{1}{1+\exp(-2f)}$， 
$$\begin{array}{rl}
likelihood(Y,p) & = Y\log p + (1-Y)\log(1-p)\\
     & = \frac{1+y}{2}\log p + \frac{1-y}{2}\log(1-p) \\
     & = \frac{1+y}{2}\log\frac{1}{1+\exp(-2f)} + \frac{1-y}{2}\log \frac{\exp(-2f)}{1+ \exp{(-2f)}}\\
     & = \frac{1+y}{2}\log\frac{1}{1+\exp(-2f)} + \frac{1-y}{2}\log \frac{1}{1+ \exp{(2f)}}\\
     & = -\frac{1+y}{2}\log\big(1+e^{-2f}\big) - \frac{1-y}{2}\log\big({1+ e^{2f}}\big)\\
     & = -\log(1-e^{-2yf}), ~~~(y=-1~or ~1)
\end{array}$$

目标是极大化似然，等价于极小化负的似然，即极小化的损失函数：$\min ~L(y, f(x)) = \min ~\log(1-e^{-2yf})$

此时，$$f^*(x) = \arg\min\limits_{f(x)} E_{y|x}\bigg(\log(1+e^{-2yf(x)})\bigg) = \frac{1}{2}\log\frac{P(y=1|x)}{P(y=-1|x)}$$
为理论上$f(x)$的最优的映射。

证明：因为$E(\log(1+e^{-2yf(x)})) = P(y=1|x)\log(1+e^{-2f}) + P(y=-1|x)\log(1+e^{2f})$

所以
$$
\begin{array}{rl}
\frac{\partial E(\log(1+e^{-2yf}))}{\partial f} & = P(y=1|x)\frac{-2e^{-2f}}{1+e^{-2f}} + P(y=-1|x)\frac{2e^{2f}}{1+e^{2f}} \\
                                                & \propto P(y=1|x)\frac{-1}{1+e^{2f}} + P(y=-1|x)\frac{e^{2f}}{1+e^{2f}} \\
                                                & \propto -P(y=1|x) + e^{2f}P(y=-1|x) \\
                                                & = 0
\end{array}
$$
所以$f^* = \frac{1}{2}\log\frac{P(y=1|x)}{P(y=-1|x)}$

参考资料：
- http://statweb.stanford.edu/~tibs/book/chap14.pdf Page10
- http://docs.salford-systems.com/GreedyFuncApproxSS.pdf Page8

In [9]:
for iter_m in range(1, max_iter+1): # Chao: 用决策树拟合的步数 m
    subset = train_data
    if 0 < sample_rate < 1:
        # Chao： 这里只选择80%的Id来构造决策树。
        # Chao：未来，剩下20%的Id只是通过这个构造好的决策树的leafnodes得到取值
        subset = sample(subset, int(len(subset)*sample_rate))
    # 用损失函数的负梯度作为回归问题提升树的残差近似值
    residual = loss.compute_residual(dataset, subset, f)
    leaf_nodes = []
    targets = residual
    tree = construct_decision_tree(dataset, subset, targets, 0, leaf_nodes, max_depth, loss, split_points)
    trees[iter_m] = tree
    loss.update_f_value(f, tree, leaf_nodes, subset, dataset, learn_rate) # Chao：更新每个Id（样本点）的值
    if isinstance(loss, RegressionLossFunction):
        # todo 判断回归的效果
        pass
    else:
        train_loss = compute_loss(dataset, train_data, f)
#         print("iter_m%d : train loss=%f" % (iter_m,train_loss))
        print(iter_m, train_loss)


1
1 0.3897065340426447
1
2 0.25475566741135086
1
3 0.16685470123077706
1
4 0.12415213494532737
1
5 0.09177197718441253
1
6 0.06470819435044439
1
7 0.055316774649429835
1
8 0.04360475947332747
1
9 0.03692395239087183
1
10 0.029970000056618883
1
11 0.026079538760605498
1
12 0.020691391394846177
1
13 0.016758166111156315
1
14 0.014348610847863588
1
15 0.011982645998606168
1
16 0.009691216866636843
1
17 0.008757701950884659
1
18 0.007690325471064932
1
19 0.006536597394716198
1
20 0.00548404731063743


In [10]:
# iter_m = 1

# subset = train_data
# if 0 < sample_rate < 1:
#     subset = sample(subset, int(len(subset)*sample_rate))
# residual = loss.compute_residual(dataset, subset, f)
# leaf_nodes = []
# targets = residual
# tree = construct_decision_tree(dataset, subset, targets, 0, leaf_nodes, max_depth, loss, split_points)
# trees[iter_m] = tree
# loss.update_f_value(f, tree, leaf_nodes, subset, dataset, learn_rate) # Chao：更新每个Id（样本点）的值

In [11]:
# # the content of compute_loss
# for id in dataset.get_instances_idset():
#     instance = dataset.get_instance(id)
#     f_values = f[id]
#     print(id, f_values)
#     exp_values = {}
#     for label in f_values:
#         exp_values[label] = exp(f_values[label])
#         print(exp_values[label])

In [12]:
#     for label in f_values:
#         exp_values[label] = exp(f_values[label])
#     probs = {}
#     for label in f_values:
#         probs[label] = exp_values[label]/sum(exp_values.values())
#     total_loss -= log(probs[instance["label"]])
# # return total_loss/dataset.size()